In [25]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

In [26]:
img_width, img_height = 224, 224

In [27]:
train_data_dir = "D:\ML\VehicalDataset\VMMR3"
validation_data_dir = "D:\ML\VehicalDataset\VMMRval"
batch_size =20

In [28]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 17243 images belonging to 38 classes.
Found 7306 images belonging to 38 classes.


In [29]:
nb_train_samples = 17243
nb_validation_samples = 7306 

In [31]:
base = applications.ResNet50(weights='imagenet', include_top=False)
x = base.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(38, activation='softmax')(x)
transfer = Model(inputs=base.input, outputs=predictions)
for layer in transfer.layers[:20]:
    layer.trainable=False
for layer in transfer.layers[20:]:
    layer.trainable=True

D:\Anaconda\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [32]:
transfer.compile(loss='categorical_crossentropy',optimizer=optimizers.SGD(lr= 0.0001, momentum=0.9),
              metrics=['accuracy'])

In [33]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18002272409389767133
]


In [34]:
from datetime import datetime
import keras.callbacks
logdir="ResNetLogs"
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
import tensorflow as tf
history_pretrained = transfer.fit_generator(train_generator,
    epochs=10, shuffle = True, verbose = 1, 
    validation_data = validation_generator,steps_per_epoch=862,validation_steps=365)

Epoch 1/10
862/862 [==============================] - 6613s 8s/step - loss: 3.6467 - acc: 0.0462 - val_loss: 5.5989 - val_acc: 0.0236
Epoch 2/10
862/862 [==============================] - 6488s 8s/step - loss: 3.3853 - acc: 0.1115 - val_loss: 5.4721 - val_acc: 0.0264
Epoch 3/10
862/862 [==============================] - 6445s 7s/step - loss: 3.0776 - acc: 0.1961 - val_loss: 4.8309 - val_acc: 0.0280
Epoch 4/10
862/862 [==============================] - 6444s 7s/step - loss: 2.7673 - acc: 0.2782 - val_loss: 5.0246 - val_acc: 0.0276
Epoch 5/10
862/862 [==============================] - 6752s 8s/step - loss: 2.4841 - acc: 0.3427 - val_loss: 4.9658 - val_acc: 0.0270
Epoch 6/10
861/862 [============================>.] - ETA: 6s - loss: 2.2312 - acc: 0.4046 